การควบคุมเหมาะที่สุดและการเรียนรูู้เสริมกำลัง -- ดร.วโรดม ตู้จินดา

# 5 แนววิถีเหมาะที่สุดแบบไม่เป็นเชิงเส้น

จากบทที่ 4 เราได้ศึกษาการโปรแกรมพลวัตขั้นพื้นฐานสำหรับระบบเชิงเส้นที่เป็นคอนเวกซ์ โดยยกตัวอย่างตัวควบคุม MPC 
แบบคอนเวกซ์ เมื่อจัดรูปเป็นปัญหา QP สามารถหาคำตอบเหมาะที่สุดได้ภายในคาบเวลาที่สั้น 
ทำให้ปัจจุบันมีการใช้งานอย่างแพร่หลายในหุ่นยนต์และอุตสาหกรรมยานยนต์

ในบทนี้จะขยายไปยังปัญหาทั่วไปที่พลวัตไม่เป็นเชิงเส้น 
และไม่คอนเวกซ์ แต่ก่อนจะเข้าสู่เนื้อหา เราเน้นย้ำว่าหากการประมาณค่าเป็นพลวัตเชิงเส้นทำงานได้ดี 
การใช้การควบคุม LQR หรือ MPC จะรับประกันการลู่เข้าสู่คำตอบภายในเวลาที่กำหนดได้ 
และมีขั้นตอนการแก้ปัญหาเพื่อหาคำตอบอย่างเป็นระบบ ในขณะที่วิธีการไม่เป็นเชิงเส้นที่จะได้ศึกษาต่อไปนี้ 
ไม่มีการรับประกันการลู่เข้าสู่ค่าเหมาะที่สุด โดยอาจลู่เข้าสู่ค่าต่ำสุดเฉพาะที่ หรือเราอาจจำเป็นต้องหยุดอัลกอริทึมก่อนกำหนดเพื่อได้เอาต์พุตตัวควบคุมภายในคาบเวลาที่กำหนด 
เรียกว่าเป็นคำตอบเกือบเหมาะที่สุด (sub-optimal) อย่างไรก็ตาม วิธีหาแนววิถีเหมาะที่สุดแบบไม่เป็นเชิงเส้น (nonlinear optimal trajectory) สามารถใช้งานได้ดี เพียงต้องใช้ความพยายามมากขึ้น 

## 5.1 การหาแนววิถีเหมาะที่สุดโดยอ้อม

วิธีการแรกที่จะกล่าวถึงเรียกว่า การโปรแกรมพลวัติเชิงอนุพันธ์ (differential dynamic programming เรียกย่อว่า DDP) [1,2]
ที่มีความคล้ายกับวิธีคุมค่ากำลังสองเชิงเส้นทำซ้ำ (iterative linear quadratic regulator เรียกย่อว่า iLQR) โดยแตกต่างกันในรายละเอียดปลีกย่อย วิธีการนี้จัดอยู่ในประเภทการหาแนววิถีเหมาะที่สุดโดยอ้อม (indirect trajectory optimization) โดยหากเปรียบเทียบกับวิธีการยิงโดยอ้อมในบทที่ 3 ซึ่งเป็นการลดค่าเกรเดียนต์ หรือรูปแบบอันดับหนึ่ง  วิธีการนี้จะมีพื้นฐานอยู่บนวิธีนิวตันคือมีรูปแบบอันดับสอง

**หมายเหตุ :** สัญกรณ์ที่ใช้จะอ้างอิงตามรายวิชา [3] ส่วนที่แตกต่างจากบทที่ 4 คือใช้ $l_k(x_k,u_k)$ และ $l_N(x_N)$ แทนมูลค่าขั้นและมูลค่าขั้นสุดท้ายตามลำดับ $V_k(x)$ แทนมูลค่ารายทาง และ $S_k(x,u)$ แทนฟังก์ชันตัวกระทำ-มูลค่า​ (action-value function นิยมเรียกว่าฟังก์ชัน $\mathcal{Q}$) 

รูปแบบปัญหาการหาแนววิถีเหมาะที่สุดแบบไม่เป็นเชิงเส้นเขียนได้เป็นดังนี้

$$
\begin{align*}
\underset{\substack{x_{1:N} \\ u_{1:N-1}}}{min} \; J &=  \sum_{k=1}^{N-1} \; l_k(x_k,u_k) + l_N(x_N) \\
s.t. \;\; x_{k+1} &= f(x_k,u_k) \\
x_k &\in \mathcal{X}_k \\
u_k &\in \mathcal{U}_k \tag{5.1}
\end{align*}
$$

โดยที่พลวัต $f(x_k,u_k)$ ไม่เป็นเชิงเส้น และเงื่อนไขบังคับสำหรับ $x_k, u_k$ ไม่เป็นคอนเวกซ์ โดยทั่วไปจะตั้งสมมุติฐานว่าฟังก์ชันมูลค่าและเงื่อนไขบังคับเป็นแบบอย่างน้อย $\mathcal{C}^2$ คือสามารถหาอนุพันธ์ได้ถึงอันดับสอง

วิธีการ DDP มีพื้นฐานอยู่บนวิธี DP แบบประมาณค่า โดยใช้การกระจายเทเลอร์อันดับสองของฟังก์ชันมูลค่ารายทางใน DP เพื่อคำนวณขั้นนิวตัน การทำงานสามารถลู่เข้าได้เร็ว หรืออาจหยุดก่อนหน้าในการใช้งานแบบเรียลไทม์

จากฟังก์ชันมูลค่ารายทาง สามารถประมาณค่าโดยกระจายเทเลอร์ได้ดังนี้

$$
V_k(x + \Delta x) \approx V_k(x) + p_k^T\Delta x + \frac{1}{2}\Delta x^T P_k \Delta x \tag{5.2}
$$

โดยนิยาม
$$
p_k = \nabla_x l_N(x) \tag{5.3}
$$
$$
P_N = \nabla_{xx}^2 l_N(x) \tag{5.4}
$$

ต่อมาพิจารณาฟังก์ชันตัวกระทำ-มูลค่า 
$$
S_k(x,u) = l_k(x,u) + V_{k+1}(f(x,u)) \tag{5.5}
$$

เมื่อประมาณค่าโดยกระจายเทเลอร์จะได้เป็น
$$
S_k(x+\Delta x, u+\Delta u) \approx S_k(x,u) + 
\left[\begin{array}{c}
g_x \\
g_u
\end{array}\right]^T
\left[\begin{array}{c}
\Delta x \\
\Delta u
\end{array}\right] + \frac{1}{2}
\left[\begin{array}{c}
\Delta x \\
\Delta u
\end{array}\right]^T 
\left[\begin{array}{cc}
G_{xx} & G_{xu} \\
G_{ux} & G_{uu}
\end{array}\right]
\left[\begin{array}{c}
\Delta x \\
\Delta u
\end{array}\right] \tag{5.6}
$$
โดยเมทริกซ์ในพจน์ด้านขวา (เฮสเชียน) เป็นแบบสมมาตร $G_{ux} = G_{xu}^T$ 

ต้องการแทนค่าย้อนหลังในวิธี DP มูลค่ารายทางย้อนหลังหนึ่งขั้นเวลามีค่าเป็น

$$
\begin{align*}
V_{k-1}(x) = \underset{\Delta u}{min} \;\Big[ & S_{k-1}(x,u) + g_x^T\Delta x + g_u^T \Delta u \\
&+ \frac{1}{2}\Delta x^TG_{xx}\Delta x + \frac{1}{2}\Delta u^TG_{uu}\Delta u \\ 
&+ \frac{1}{2}\Delta x^TG_{xu}\Delta u + \frac{1}{2}\Delta u^TG_{ux}\Delta x \Big] \tag{5.7} 
\end{align*}
$$

หาเกรเดียนต์เทียบกับ $\Delta u$ และให้เท่ากับศูนย์ 
$$
\nabla_{\Delta u}[V_{k-1}(x)] = g_u + G_{uu}\Delta u + G_{ux}\Delta x = 0 \tag{5.8}
$$

จาก (5.8) สามารถหาคำตอบของ $\Delta u$ เป็นฟังก์ชันของพจน์ที่เหลือโดยการจัดรูปสมการได้เป็น
$$
\begin{align*}
\Delta u_{k-1} &= - G_{uu}^{-1}g_u - G_{uu}^{-1}G_{ux}\Delta x \\
&\triangleq -d_{k-1} - K_{k-1} \Delta x \tag{5.9}
\end{align*}
$$

โดยนิยาม $d_{k-1}$ และ $K_{k-1}$ เป็นพจน์ป้อนข้างหน้า​ (feedforward) และป้อนกลับ (feedback) ตามลำดับ แทนค่า (5.9) ลงใน $S_{k-1}$ เพื่อได้ $V_{k-1}(x + \Delta x)$ 
$$
\begin{align*}
V_{k-1}(x + \Delta x) & \approx V_{k-1}(x) + g_x^T\Delta x + g_u^T (-d_{k-1} - K_{k-1} \Delta x) \\
&+ \frac{1}{2}\Delta x^TG_{xx}\Delta x + \frac{1}{2}(-d_{k-1} - K_{k-1} \Delta x)^TG_{uu}(-d_{k-1} - K_{k-1} \Delta x) \\ 
&+ \frac{1}{2}\Delta x^TG_{xu}(-d_{k-1} - K_{k-1} \Delta x) + \frac{1}{2}(-d_{k-1} - K_{k-1} \Delta x)^TG_{ux}\Delta x  \tag{5.10} 
\end{align*}
$$

ดังนั้น $P_{k-1}$ และ $p_{k-1}$ ที่นิยามใน (5.2) จะได้เป็นดังนี้
$$
P_{k-1} = G_{xx} + K_{k-1}^TG_{uu}K_{k-1} - G_{xu}K_{k-1} - K_{k-1}^TG_{ux} \tag{5.11}
$$
$$
p_{k-1} = g_x - K_{k-1}g_u + K_{k-1}^TG_{uu}d_{k-1} - G_{xu}d_{k-1} \tag{5.12}
$$


## บรรณานุกรม

1. D.Q. Mayne. "A second-order gradient method of optimizing non-linear discrete time systems". 
Int J Control. 3: 85–95. 1966.

2. D.Q. Mayne and D.H Jacobson. Differential dynamic programming. New York: American Elsevier 
Pub. Co. 1970.

3. Z. Manchester et.al. [16-745 Optimal Control & Reinforcement Learning, 
Course materials](https://optimalcontrol.ri.cmu.edu/#learning-resources), Carnegie Mellon University. 2025.



<div align="center">
<img src="https://raw.githubusercontent.com/dewdotninja/sharing-github/refs/heads/master/dewninja_logo50.jpg" alt="dewninja"/>
</div>
<div align="center">dew.ninja 2025</div>